In [1]:
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf
import statsmodels.api as sm

import matplotlib.pyplot as plt

Question 1

Use instrumental variables to find the effect of X on Z in homework_13.1.csv. Which of these is the closest? 

In [2]:
# Load data
df = pd.read_csv('homework_13.1.csv')

In [3]:
Xhat = sm.OLS(df["X"], sm.add_constant(df["Y"])).fit().fittedvalues

# 2nd stage: Z ~ 1 + X̂  → IV estimate is coef on X̂
iv = sm.OLS(df["Z"], sm.add_constant(Xhat)).fit(cov_type="HC1")
print(iv.params)            # effect of X on Z is iv.params[1]

const    0.361379
0        0.222003
dtype: float64


## Question 2

Use two-stage least squares to find the effect of X2 on Y2 in homework_13.2.csv. Which of these is the closest? 

In [5]:
# Load data
df = pd.read_csv('homework_13.2.csv')

In [7]:
# First stage: X2 ~ 1 + Z2  → get fitted X2 (X2_hat)
X2_hat = sm.OLS(df["X2"], sm.add_constant(df["Z2"])).fit().fittedvalues

# Second stage: Y2 ~ 1 + X2_hat  → IV estimate is coef on X2_hat
iv = sm.OLS(df["Y2"], sm.add_constant(X2_hat)).fit(cov_type="HC1")
print("2SLS coef on X2 (manual):", iv.params.iloc[1])

2SLS coef on X2 (manual): -2.502618970813034


## Question 3

In homework_13.3.csv, which are the compliers? Recall that for a complier, changing the Z value from 0 to 1 would increase the likely value of X. Here, ZW_int = Z3 * W3. All confounders are given in this case, so you can use linear regression to understand the situation. (2SLS is not necessary.) 

In [8]:
# Load data
df = pd.read_csv('homework_13.3.csv')

In [ ]:
# First-stage regression: X3 ~ Z3 + W3 + ZW_int
X = sm.add_constant(df[["Z3","W3","ZW_int"]])
model = sm.OLS(df["X3"], X).fit()

# Predict X3 if Z3 = 1 and if Z3 = 0 (holding W3 fixed)
df_pred = df.copy()
df1 = df.copy()
df1["Z3"] = 1
df1["ZW_int"] = df1["Z3"] * df1["W3"]

df0 = df.copy()
df0["Z3"] = 0
df0["ZW_int"] = df0["Z3"] * df0["W3"]

pred1 = model.predict(sm.add_constant(df1[["Z3","W3","ZW_int"]]))
pred0 = model.predict(sm.add_constant(df0[["Z3","W3","ZW_int"]]))

# Change in predicted X when Z changes from 0 to 1
df["delta_X"] = pred1 - pred0

# Mark compliers: delta_X > 0
df["complier"] = df["delta_X"] > 0

print(df[["Z3","X3","W3","delta_X","complier"]])

KeyError: "['const'] not in index"

## Question 4

```python
Z4 = np.random.normal(0, 1, num)
W4 = np.random.normal(0, 1, num)
ZW_int = W4 * Z4
X4 = Z4 + W4 - 2 * ZW_int + np.random.normal(0, 1, num)
Y4 = W4 + X4 - Z4 + np.random.normal(0, 1, num)
```

Is the instrument Z4 valid? If not, which of the following is a reason that it is invalid? 